In [ ]:
import pandas as pd
import os

In [ ]:

empresas = ["ABEV3", "AZUL4", "BTOW3", "B3SA3", "BBSE3", "BRML3", "BBDC4", "BRAP4", "BBAS3", "BRKM5", "BRFS3", "BPAC11", "CRFB3", "CCRO3", "CMIG4", "HGTX3", "CIEL3", "COGN3", "CPLE6", "CSAN3", "CPFE3", "CVCB3", "CYRE3", "ECOR3", "ELET6", "EMBR3", "ENBR3", "ENGI11", "ENEV3", "EGIE3", "EQTL3", "EZTC3", "FLRY3", "GGBR4", "GOAU4", "GOLL4", "NTCO3", "HAPV3", "HYPE3", "IGTA3", "GNDI3", "ITSA4", "ITUB4", "JBSS3", "JHSF3", "KLBN11", "RENT3", "LCAM3", "LAME4", "LREN3", "MGLU3", "MRFG3", "BEEF3", "MRVE3", "MULT3", "PCAR3", "PETR4", "BRDT3", "PRIO3", "QUAL3", "RADL3", "RAIL3", "SBSP3", "SANB11", "CSNA3", "SULA11", "SUZB3", "TAEE11", "VIVT3", "TIMS3", "TOTS3", "UGPA3", "USIM5", "VALE3", "VVAR3", "WEGE3", "YDUQ3"]

fundamentos = {}
arquivos = os.listdir('balancos')

print(len(empresas))

In [ ]:
for arquivo in arquivos:
    nome = arquivo[-9:-4]
    if '11' in nome:
        nome = arquivo[-10:-4]
        print(nome)
    if nome in empresas:
        #pegar o balanço da empresa
        balanco = pd.read_excel(f'balancos/{arquivo}', sheet_name=0)
        #1° coluna e colocar titulo com o nome da empresa
        balanco.iloc[0, 0] = nome
        #pegar 1° linha e tornar cabeçalho
        balanco.columns = balanco.iloc[0]
        balanco = balanco[1:]
        #pegar 1° coluna (ja com nome da empresa) como indice
        balanco = balanco.set_index(nome)
        
        
        #DRE
        
        dre = pd.read_excel(f'balancos/{arquivo}', sheet_name=1)
        #1° coluna e colocar titulo com o nome da empresa
        dre.iloc[0, 0] = nome
        #pegar 1° linha e tornar cabeçalho
        dre.columns = dre.iloc[0]
        dre = dre[1:]
        #pegar 1° coluna (ja com nome da empresa) como indice
        dre = dre.set_index(nome)
        fundamentos[nome]=pd.concat([balanco,dre])











In [ ]:
cotacoes_df = pd.read_excel("Cotacoes.xlsx")
cotacoes = {}
for empresa in cotacoes_df["Empresa"].unique():
    cotacoes[empresa] = cotacoes_df.loc[cotacoes_df['Empresa']==empresa, :]

display(cotacoes['ABEV3'])

In [ ]:
for empresa in empresas:
    if cotacoes[empresa].isnull().values.any():
        cotacoes.pop(empresa)
        fundamentos.pop(empresa)

empresas = list(cotacoes.keys())
print(len(empresas))

In [ ]:
#cotaçoes data p/ indice
#fundamentos:
    #trocar linhas / colunas
    #tratar as datas para formato de data do python
    #juntar os fundamentos com a coluna Adj Close das cotacoes

for empresa in fundamentos: 
    tabela = fundamentos[empresa].T#.T troca linha por coluna
    tabela.index = pd.to_datetime(tabela.index, format="%d/%m/%Y")
    tabela_cotacoes = cotacoes[empresa].set_index("Date")
    tabela_cotacoes = tabela_cotacoes[["Adj Close"]]
    
    tabela = tabela.merge(tabela_cotacoes, right_index=True, left_index=True)
    tabela.index.name = empresa
    fundamentos[empresa] = tabela
display(fundamentos['ABEV3'])
    

In [ ]:
#REMOVER DA ANALISE TABELAS QUE TEM COLUNAS DIFERENTES

colunas = list(fundamentos["ABEV3"].columns)

for empresa in empresas:
    if set(colunas) != set(fundamentos[empresa].columns):
        fundamentos.pop(empresa)
print(len(fundamentos))


In [ ]:
texto_colunas = ";".join(colunas)

colunas_modificadas = []
for coluna in colunas:
    if colunas.count(coluna) == 2 and coluna not in colunas_modificadas:
        texto_colunas = texto_colunas.replace(";" + coluna + ";", ";" + coluna + "_1;", 1)
        colunas_modificadas.append(coluna)
colunas = texto_colunas.split(';')


In [ ]:
#add colunas na  tabela
for empresa in fundamentos:
    fundamentos[empresa].columns = colunas

In [ ]:
#analisar valores vazios

valores_vazios = dict.fromkeys(colunas, 0)
total_linhas = 0

for empresa in fundamentos:
    tabela = fundamentos[empresa]
    total_linhas += tabela.shape[0]
    for coluna in colunas:
        qtd_vazios = pd.isnull(tabela[coluna]).sum()
        valores_vazios[coluna] += qtd_vazios








In [ ]:
remover_colunas = []
for coluna in valores_vazios:
    if valores_vazios[coluna] > 50:
        remover_colunas.append(coluna)
for empresa in fundamentos:
    fundamentos[empresa] = fundamentos[empresa].drop(remover_colunas, axis=1)
    fundamentos[empresa] = fundamentos[empresa].ffill()
    #teste
    